In [907]:
import pandas as pd
import os

from datetime import datetime, date, timedelta
from dateutil.relativedelta import *

import numpy as np
import math

import calendar
from pathlib import Path


import requests
import urllib.parse

import quandl
quandl.ApiConfig.api_key = "ca9bMvozbsfJ41tQECsN"

pd.set_option('max_columns',250)
pd.set_option('max_rows',100)

In [908]:
def file_list(ticker):

    statement_folder = f"../data/timeseries/{ticker}/"
    file_list = []
    count = 0 
    for dirname, _, filenames in os.walk(f"{statement_folder}"):
        for filename in filenames:
            file_list.append(os.path.join(dirname,filename))
            print(f"{count}: {filename}")
            count +=1
    return file_list

In [909]:
def retrieve_df(list_num):
    print(file_list[list_num])
    df = pd.read_csv(file_list[list_num],index_col=[0,1])
    return df

In [910]:
def fetch_quandl(ticker):
    df_quandl_tmp = quandl.get_table("ZACKS/FC",ticker=ticker,per_type='Q')
    if 'per_end_date' in df_quandl_tmp:
        df_quandl_tmp.index = df_quandl_tmp['per_end_date'].astype(str)
        del(df_quandl_tmp['per_end_date'])
    return df_quandl_tmp

In [913]:
def download_stock_row_statements(ticker,data_path):
    stockrow_folder = f'{data_path}stockrow/{ticker}/'
    Path(stockrow_folder).mkdir(parents=True, exist_ok=True)

    for statement in ['Income Statement','Cash Flow','Balance Sheet']:
        url = f'https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=Q&section={urllib.parse.quote(statement)}&sort=desc'
        try:
            r = requests.get(url, allow_redirects=True)
            open(f'{stockrow_folder}/{statement}.xlsx', 'wb').write(r.content)
        except: 
            print(f'Failed to download {statement} for {ticker}')

def load_stockrow_statement(ticker,data_path,statement_type):
    statement_folder = f'{data_path}stockrow/{ticker}/'
    df_comparison = pd.read_excel(f'{statement_folder}{statement_type}.xlsx',index_col=[0])
    df_comparison.columns = pd.DatetimeIndex(df_comparison.columns)
            
def list_of_rounded_months(date_list):
        rounded_date_list = []
        
        for date_str in date_list:

            rounded_date = date_round(date_str)
            rounded_date_list.append(rounded_date)
                
        return rounded_date_list
        

def date_round(date_str):
    year = int(date_str[0:4])
    
    month = int(date_str[5:7])
    day = int(date_str[8:10])

    if round(day/calendar.monthrange(year,month)[1],0) == 1:
        rounded_date = (date(year, month, 1) + relativedelta(months=+1) - timedelta(days=1))
    else:
        rounded_date = (date(year, month, 1) - timedelta(days=1))

    return rounded_date

def adjust_timeseries_for_match(df_timeseries,df_comparison):
    df_timeseries = df_timeseries.loc[df_timeseries[df_timeseries.columns.max()].notna(),:] #delete rows with na for latest period
    rounded_date_list = list_of_rounded_months(df_timeseries.columns)
    df_timeseries.columns = pd.DatetimeIndex(rounded_date_list)
    #df_timeseries.columns = df_timeseries.columns)

    
    #df_timeseries.index = df_timeseries.index
    #df_timeseries = df_timeseries.loc[:,df_timeseries.columns.isin(df_comparison.columns)]/1000000
    return df_timeseries

def match_statement(df_timeseries,df_comparison):
    tmp_mapping_dict = {}

    for comparison_row in df_comparison.index:
        
        #print(f"To Match: {comparison_row}")

        
        for timeseries_row in df_timeseries.index:
            #print(timeseries_row)
            if (timeseries_row[1][:8] == 'us-gaap_') and ('___' not in timeseries_row[1]):
                
                

                
                s_timeseriesrow_not_na = df_timeseries.loc[timeseries_row,df_timeseries.columns.isin(df_comparison.columns)].dropna() 
                s_comparisonrow_matchdates = df_comparison.loc[comparison_row,df_comparison.columns.isin(s_timeseriesrow_not_na.index)]
                s_match = s_timeseriesrow_not_na == s_comparisonrow_matchdates
                match_pct = s_match.sum()/s_match.count()

                if match_pct == 1:
                    #print(f"MATCH: {timeseries_row[0]}")
                    
                    if timeseries_row[0] not in tmp_mapping_dict:
                        tmp_mapping_dict[timeseries_row[0]] = []
                    #else:
                        #print(f'{timeseries_row[0]} already has this value: {tmp_mapping_dict[timeseries_row[0]]}')
                        
                    tmp_mapping_dict[timeseries_row[0]].append(comparison_row)
    
    return tmp_mapping_dict


def rename_index_labels(df_timeseries,matched_mapping_dict,matched_statement):
    df_timeseries = pd.read_csv(os.path.join(timeseries_statement_folder,matched_statement),index_col=[0,1])
    df_timeseries = df_timeseries.rename(index=matched_mapping_dict,level=0)
    return df_timeseries

def match_statement_and_rename_labels(ticker, statement,data_path):
    

    df_comparison = load_stockrow_data(ticker,data_path,statement)
    #print(f"{statement} has {len(df_comparison)} rows")
    
    max_overlap = 0
    
    for statement_csv in os.listdir(timeseries_statement_folder):
        df_timeseries = pd.read_csv(f'{timeseries_statement_folder}{statement_csv}',index_col=[0,1])
        
        df_timeseries = adjust_timeseries_for_match(df_timeseries,df_comparison)
        tmp_mapping_dict = match_statement(df_timeseries,df_comparison)

        if len(tmp_mapping_dict.keys()) > max_overlap:
            max_overlap = len(tmp_mapping_dict.keys())
            matched_statement = statement_csv 
            matched_mapping_dict = tmp_mapping_dict

    for key in matched_mapping_dict:
        if len(matched_mapping_dict[key]) > 1:
            print(f"{key}: {', '.join(matched_mapping_dict[key])}")
            #print(matched_mapping_dict[key])
    
    
    #df_timeseries = rename_index_labels(df_timeseries,matched_mapping_dict,matched_statement)

        
    return df_timeseries

def fill_missing_from_stockrow(df,ticker,data_path):

    df_stockrow = pd.read_excel(f'{data_path}stockrow/{ticker}/{statement}.xlsx',index_col=[0])

    month_end_columns = list_of_rounded_months(df.columns)
    #print(month_end_columns)
    #[datetime.strptime(f"{date_col[0:4]}-{date_col[5:7]}-{calendar.monthrange(int(date_col[0:4]),int(date_col[5:7]))[1]}",'%Y-%m-%d') for date_col in df[statement].columns]

    for row in df.loc[df.index.get_level_values(0).isin(df_stockrow.index),:].index:
        #print(row)
        for column in df.loc[:,pd.DatetimeIndex(month_end_columns).isin(df_stockrow.columns)]:
            #print(column)
            rounded_column = date_round(column)
            if (math.isnan(df.loc[row,column])) and (row[0] in df_stockrow.index) and (rounded_column in df_stockrow.columns):
                df.loc[row,column] = df_stockrow.loc[row[0],rounded_column]
                print(f"Filled in [{row[0]},{column}] from stockrow")
    
    return df


In [914]:
data_path = '../data/'
ticker_list = ['AAPL','AXP','BA','CAT','CVX','DD','GS','IBM','INTC','JPM','KO','MMM','MRK','MSFT','PFE','PG','CSCO', 'DIS','GE','HD','JNJ','MCD', 'NKE']

for ticker in ticker_list[:1]:
    print(f'______{ticker}______')
    
    download_stock_row_statements(ticker,data_path)
    timeseries_statement_folder = f'{data_path}timeseries/{ticker}/Statement/'
    
    df = {}

    for statement in ['Income Statement','Cash Flow','Balance Sheet']:
    #for statement in ['Income Statement']:
        print(f'____{statement}____')
        df[statement] = match_statement_and_rename_labels(ticker, statement,data_path)
        #df[statement] = fill_missing_from_stockrow(df[statement],ticker,data_path)
    
    
#df['Income Statement']

______AAPL______
____Income Statement____
Operating income: Operating Income, EBIT
Net income: Income after Tax, Net Income Common, Income from Continuous Operations, Consolidated Net Income/Loss
____Cash Flow____
____Balance Sheet____
Total shareholders’ equity: Common Equity (Total), Shareholders Equity (Total)


In [ ]:
df[statement]

In [831]:

data_path = '../data/'
ticker = 'AAPL'
stockrow_folder = f'{data_path}stockrow/{ticker}/'
statement = 'Income Statement'


In [828]:
df[statement]

,,2020-03-28,2019-12-28,2019-09-28,2019-06-29,2019-03-30,2018-12-29,2018-09-29,2018-06-30,2018-03-31,2017-12-30,2017-09-30,2017-07-01,2017-04-01,2016-12-31,2016-09-24,2016-06-25,2016-03-26,2015-12-26,2015-09-26,2015-06-27,2015-03-28,2014-12-27,2014-09-27,2014-06-28,2014-03-29,2013-12-28,2013-09-28,2013-06-29,2013-03-30,2012-12-29,2012-09-29,2012-06-30,2012-03-31,2011-12-31,2011-09-24,2011-06-25,2011-03-26,2010-12-25,2010-09-25,2010-06-26,2010-03-27,2009-12-26,2009-09-26,2009-06-27,2009-03-28,2008-12-27,2008-09-27,2008-06-28,2007-09-29
label,xbrl_tag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Revenue,us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,5.831300e+10,9.181900e+10,6.404000e+10,5.380900e+10,5.801500e+10,8.431000e+10,6.290000e+10,5.326500e+10,6.113700e+10,8.829300e+10,5.257900e+10,4.540800e+10,5.289600e+10,7.835100e+10,4.685200e+10,4.235800e+10,5.055700e+10,7.587200e+10,5.150100e+10,4.960500e+10,5.801000e+10,7.459900e+10,4.212300e+10,3.743200e+10,4.564600e+10,5.759400e+10,3.747200e+10,3.532300e+10,4.360300e+10,5.451200e+10,3.596600e+10,3.502300e+10,3.918600e+10,4.633300e+10,2.827000e+10,2.857100e+10,2.466700e+10,2.674100e+10,2.034300e+10,1.570000e+10,1.349900e+10,1.568300e+10,7.236000e+09,8.337000e+09,9.084000e+09,1.188000e+10,NaN,7.464000e+09,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Product [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,4.496500e+10,7.910400e+10,5.152900e+10,4.235400e+10,4.656500e+10,7.343500e+10,5.230100e+10,4.309500e+10,5.128700e+10,7.916400e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Service [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,1.334800e+10,1.271500e+10,1.251100e+10,1.145500e+10,1.145000e+10,1.087500e+10,1.059900e+10,1.017000e+10,9.850000e+09,9.129000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Revenue,us-gaap_costofgoodsandservicessold,3.594300e+10,5.660200e+10,3.972700e+10,3.358200e+10,3.619400e+10,5.227900e+10,3.881600e+10,3.284400e+10,3.771500e+10,5.438100e+10,3.264800e+10,2.792000e+10,3.230500e+10,4.817500e+10,2.903900e+10,2.625200e+10,3.063600e+10,4.544900e+10,3.095300e+10,2.992400e+10,3.435400e+10,4.485800e+10,2.611400e+10,2.269700e+10,2.769900e+10,3.574800e+10,2.360100e+10,2.229900e+10,2.725400e+10,3.345200e+10,2.156500e+10,2.002900e+10,2.062200e+10,2.563000e+10,1.689000e+10,1.664900e+10,1.444900e+10,1.644300e+10,1.283100e+10,9.564000e+09,7.874000e+09,9.272000e+09,5.253000e+09,5.314000e+09,5.457000e+09,7.373000e+09,NaN,4.864000e+09,NaN
Cost of sales (Product [Member]),us-gaap_costofgoodsandservicessold___us-gaap_productmember,3.132100e+10,5.207500e+10,3.523800e+10,2.947300e+10,3.204700e+10,4.823800e+10,3.469700e+10,2.895600e+10,3.393600e+10,5.057500e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of sales (Service [Member]),us-gaap_costofgoodsandservicessold___us-gaap_servicemember,4.622000e+09,4.527000e+09,4.489000e+09,4.109000e+09,4.147000e+09,4.041000e+09,4.119000e+09,3.888000e+09,3.779000e+09,3.806000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,us-gaap_grossprofit,2.237000e+10,3.521700e+10,2.431300e+10,2.022700e+10,2.182100e+10,3.203100e+10,2.408400e+10,2.042100e+10,2.342200e+10,3.391200e+10,1.993100e+10,1.748800e+10,2.059100e+10,3.017600e+10,1.781300e+10,1.610600e+10,1.992100e+10,3.042300e+10,2.054800e+10,1.968100e+10,2.365600e+10,2.974100e+10,1.600900e+10,1.473500e+10,1.794700e+10,2.184600e+10,1.387100e

In [767]:
month_end_columns[0] == df_stockrow.columns[0]

True

Timestamp('2020-03-31 00:00:00')

,,2020-03-28,2019-12-28,2019-09-28,2019-06-29,2019-03-30,2018-12-29,2018-09-29,2018-06-30,2018-03-31,2017-12-30,2017-09-30,2017-07-01,2017-04-01,2016-12-31,2016-09-24,2016-06-25,2016-03-26,2015-12-26,2015-09-26,2015-06-27,2015-03-28,2014-12-27,2014-09-27,2014-06-28,2014-03-29,2013-12-28,2013-09-28,2013-06-29,2013-03-30,2012-12-29,2012-09-29,2012-06-30,2012-03-31,2011-12-31,2011-09-24,2011-06-25,2011-03-26,2010-12-25,2010-09-25,2010-06-26,2010-03-27,2009-12-26,2009-09-26,2009-06-27,2009-03-28,2008-12-27,2008-09-27,2008-06-28,2007-09-29
label,xbrl_tag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Revenue,us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,5.831300e+10,9.181900e+10,6.404000e+10,5.380900e+10,5.801500e+10,8.431000e+10,6.290000e+10,5.326500e+10,6.113700e+10,8.829300e+10,NaN,4.540800e+10,5.289600e+10,NaN,4.685200e+10,4.235800e+10,5.055700e+10,7.587200e+10,5.150100e+10,4.960500e+10,5.801000e+10,7.459900e+10,4.212300e+10,3.743200e+10,4.564600e+10,5.759400e+10,3.747200e+10,3.532300e+10,4.360300e+10,5.451200e+10,NaN,3.502300e+10,3.918600e+10,NaN,2.827000e+10,2.857100e+10,2.466700e+10,2.674100e+10,2.034300e+10,1.570000e+10,1.349900e+10,1.568300e+10,7.236000e+09,8.337000e+09,9.084000e+09,1.188000e+10,NaN,7.464000e+09,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Product [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,4.496500e+10,7.910400e+10,5.152900e+10,4.235400e+10,4.656500e+10,7.343500e+10,5.230100e+10,4.309500e+10,5.128700e+10,7.916400e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax (Service [Member])",us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,1.334800e+10,1.271500e+10,1.251100e+10,1.145500e+10,1.145000e+10,1.087500e+10,1.059900e+10,1.017000e+10,9.850000e+09,9.129000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of Revenue,us-gaap_costofgoodsandservicessold,3.594300e+10,5.660200e+10,3.972700e+10,3.358200e+10,3.619400e+10,5.227900e+10,3.881600e+10,3.284400e+10,3.771500e+10,5.438100e+10,NaN,2.792000e+10,3.230500e+10,NaN,2.903900e+10,2.625200e+10,3.063600e+10,4.544900e+10,3.095300e+10,2.992400e+10,3.435400e+10,4.485800e+10,2.611400e+10,2.269700e+10,2.769900e+10,3.574800e+10,2.360100e+10,2.229900e+10,2.725400e+10,3.345200e+10,NaN,2.002900e+10,2.062200e+10,NaN,1.689000e+10,1.664900e+10,1.444900e+10,1.644300e+10,1.283100e+10,9.564000e+09,7.874000e+09,9.272000e+09,5.253000e+09,5.314000e+09,5.457000e+09,7.373000e+09,NaN,4.864000e+09,NaN
Cost of sales (Product [Member]),us-gaap_costofgoodsandservicessold___us-gaap_productmember,3.132100e+10,5.207500e+10,3.523800e+10,2.947300e+10,3.204700e+10,4.823800e+10,3.469700e+10,2.895600e+10,3.393600e+10,5.057500e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cost of sales (Service [Member]),us-gaap_costofgoodsandservicessold___us-gaap_servicemember,4.622000e+09,4.527000e+09,4.489000e+09,4.109000e+09,4.147000e+09,4.041000e+09,4.119000e+09,3.888000e+09,3.779000e+09,3.806000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit,us-gaap_grossprofit,2.237000e+10,3.521700e+10,2.431300e+10,2.022700e+10,2.182100e+10,3.203100e+10,2.408400e+10,2.042100e+10,2.342200e+10,3.391200e+10,NaN,1.748800e+10,2.059100e+10,NaN,1.781300e+10,1.610600e+10,1.992100e+10,3.042300e+10,2.054800e+10,1.968100e+10,2.365600e+10,2.974100e+10,1.600900e+10,1.473500e+10,1.794700e+10,2.184600e+10,1.387100e+10,1.302400e+10,1.634900e+10,2.106000e+10,NaN,1.499400e+10,1.856400e+10,NaN,1.138000e+10,

In [478]:
s_timeseriesrow_not_na = df_timeseries.loc['us-gaap_revenuefromcontractwithcustomerexcludingassessedtax',df_timeseries.columns.isin(df_comparison.columns)].dropna()
s_comparisonrow_matchdates = df_comparison.loc['Revenue Growth',df_comparison.columns.isin(s_timeseriesrow_not_na.index)]
#s_match = s_timeseriesrow_not_na == s_comparisonrow_matchdates
#s_match.sum()/s_match.count()
#df_comparison[df_comparison.index.isin(


In [479]:
s_comparisonrow_matchdates


2020-03-31    0.0051
2019-12-31    0.0891
2019-09-30    0.0181
2019-06-30    0.0102
2019-03-31   -0.0511
2018-12-31   -0.0451
2018-09-30    0.1963
2018-06-30    0.1730
2018-03-31    0.1558
2017-12-31    0.1269
2016-09-30   -0.0903
2016-06-30   -0.1461
2016-03-31   -0.1285
2015-12-31    0.0171
2015-09-30    0.2226
2015-06-30    0.3252
2015-03-31    0.2709
2014-12-31    0.2953
2014-09-30    0.1241
2014-06-30    0.0597
2014-03-31    0.0469
2013-12-31    0.0565
2013-09-30    0.0419
2013-06-30    0.0086
2013-03-31    0.1127
2012-12-31    0.1765
2012-06-30    0.2258
2012-03-31    0.5886
2011-09-30    0.3897
2011-06-30    0.8198
2011-03-31    0.8273
2010-12-31    0.7051
2010-09-30    0.6665
Name: Revenue Growth, dtype: float64